In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first


    
def zeroOfLinearMonomial(monomial,x):return x-monomial


def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def primesToN(n):  # lists primes p w. 2 <= p <= n
    answer=[]
    for k in [1..n-1]:
        nk=next_prime(k)
        if (nk in answer)==False:
            if nk<n+1:
                answer=answer+[next_prime(k)]
    return answer

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]] # from 'capital-J make data file1jun21.ipynb'
    # in which normalJ takes only two arguments,
    # unlike the definition above.
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [18]:
# McKay Thompson series from table https://oeis.org/A097340/b097340.txt:
mt=[[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]
def mcKayThompson(n):
    return mt[n+1][1]

In [19]:
print mcKayThompson(100)

30470821082605141257437184


In [22]:
import pickle
Jfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
Js=pickle.load(Jfile)
Jfile.close()
Js=stripQuotationMarks(Js)
print len(Js)

201


In [21]:
import pickle
Jfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
Js=pickle.load(Jfile)
Jfile.close()
Js=stripQuotationMarks(Js)

data=[]
for k in [1..3]:
    n=Js[k][0]
    Jpoly = D(Js[k][1])
    Jnf=numericalfactor(Jpoly)
    print "======================================================================================="
    print Jpoly.factor()
    print (mt[n+1][1],Jnf,mcKayThompson(n)/Jnf-2^(6*n+6))
    data=data+[mt[n+1][1]/Jnf-2^(6*n+6)]
print;print data


(3/8) * (x^2 + 4/3)
(24, 3/8, 0)
(69/1024) * (x^4 - 8/69*x^2 - 16/23)
(276, 69/1024, 0)
(1/128) * (x - 2) * (x + 2) * (x^4 - 8/27*x^2 - 16/27)
(2048, 1/128, 0)

[0, 0, 0]


In [24]:
import pickle
Jfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
Js=pickle.load(Jfile)
Jfile.close()
Js=stripQuotationMarks(Js)

data=[]
for k in [1..3]:
    n=Js[k][0]
    Jpoly = D(Js[k][1])
    Jnf=numericalfactor(Jpoly)
    print "======================================================================================="
    print Jpoly.factor()
    print (mt[n+1][1],Jnf,mcKayThompson(n)/2^(6*n+6)-Jnf)
    data=data+[mt[n+1][1]/Jnf-2^(6*n+6)]
print;print data



(3/8) * (x^2 + 4/3)
(24, 3/8, 0)
(69/1024) * (x^4 - 8/69*x^2 - 16/23)
(276, 69/1024, 0)
(1/128) * (x - 2) * (x + 2) * (x^4 - 8/27*x^2 - 16/27)
(2048, 1/128, 0)

[0, 0, 0]


In [27]:
Jfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
Js=pickle.load(Jfile)
Jfile.close()
Js=stripQuotationMarks(Js)

data=[]
for k in [1..100]:
    n=Js[k][0]
    Jpoly = D(Js[k][1])
    Jnf=numericalfactor(Jpoly)
    data=data+[mcKayThompson(n)/2^(6*n+6)-Jnf]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
# make a table for attachment to the paper.

import pickle
Jfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
Js=pickle.load(Jfile)
Jfile.close()
Js=stripQuotationMarks(Js)

data=[]
for k in [0..9]:
    n=Js[k][0]
    Jpoly = D(Js[k][1])
    print "n: ",n
    print Jpoly.factor()
    print

n:  -1
1

n:  0
(3/8) * (x^2 + 4/3)

n:  1
(69/1024) * (x^4 - 8/69*x^2 - 16/23)

n:  2
(1/128) * (x - 2) * (x + 2) * (x^4 - 8/27*x^2 - 16/27)

n:  3
(5601/8388608) * (x - 2) * (x + 2) * (x^6 - 38732/5601*x^4 + 11312/5601*x^2 + 6464/1867)

n:  4
(3/65536) * (x - 2) * (x + 2) * (x^8 - 51968/3375*x^6 + 650144/10125*x^4 - 190976/10125*x^2 - 95488/3375)

n:  5
(23003/8589934592) * (x - 2) * (x + 2) * (x^10 - 15548948/621081*x^8 + 3737957344/16769187*x^6 - 452733568/621081*x^4 + 1217201408/5589729*x^2 + 4868805632/16769187)

n:  6
(75/536870912) * (x - 2) * (x + 2) * (x^12 - 340526584/9646875*x^10 + 14381852336/28940625*x^8 - 32414868736/9646875*x^6 + 18398435584/1929375*x^4 - 16877848576/5788125*x^2 - 33755697152/9646875)

n:  7
(1881471/281474976710656) * (x - 2) * (x + 2) * (x^14 - 97388044148/2116654875*x^12 + 51129660553424/57149681625*x^10 - 320257042164544/34289808975*x^8 + 3050057679448832/57149681625*x^6 - 2640538932296704/19049893875*x^4 + 7468858536415232/171449044875*x^2 + 271594